## Logistic Regression

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split,KFold
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,\
recall_score,roc_curve,auc

import expectation_reflection as ER
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from function import split_train_test,make_data_balance

In [2]:
np.random.seed(1)

First of all, the processed data are imported.

In [3]:
#data_list = ['1paradox','2peptide','3stigma']
data_list = np.loadtxt('data_list.txt',dtype='str')
#data_list = ['29parkinson','30paradox2','31renal','32patientcare','33svr','34newt','35pcos']

print(data_list)

['1paradox' '2peptide' '3stigma' '4nki' '5mental' '6smoking' '7anemia'
 '8language' '9coag' '10tazamia' '11hepato' '12heat' '13ef' '14cervix'
 '15heart' '16liver' '17nwosu' '18school' '19ibs' '21survival' '101kidney'
 '102breast_cancer' '103diabetes_niddk' '104diabetic_retinopathy'
 '29parkinson' '30paradox2' '31renal' '33svr' '35pcos']


In [4]:
def read_data(data_id):    
    data_name = data_list[data_id]
    print('data_name:',data_name)
    Xy = np.loadtxt('../data/%s/data_processed.dat'%data_name) 
    X = Xy[:,:-1]
    y = Xy[:,-1]

    #print(np.unique(y,return_counts=True))

    X,y = make_data_balance(X,y)

    print(np.unique(y,return_counts=True))

    X, y = shuffle(X, y, random_state=1)

    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.5,random_state = 1)
    
    sc = MinMaxScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    
    return X_train,X_test,y_train,y_test

In [5]:
def measure_performance(X_train,X_test,y_train,y_test):
    
    model = LogisticRegression(max_iter=100)

    # regularization penalty space
    penalty = ['l1','l2']
    #penalty = ['elasticnet']

    # solver
    #solver=['saga']
    solver=['liblinear']

    # regularization hyperparameter space
    #C = np.logspace(0, 4, 10)
    C = [0.001,0.1,1.0,10.0,100.0]

    # l1_ratio
    #l1_ratio = [0.1,0.5,0.9]

    # Create hyperparameter options
    #hyperparameters = dict(penalty=penalty,solver=solver,C=C,l1_ratio=l1_ratio)
    hyper_parameters = dict(penalty=penalty,solver=solver,C=C)
    
    # Create grid search using cross validation
    clf = GridSearchCV(model, hyper_parameters, cv=4, iid='deprecated')
    
    # Fit grid search
    best_model = clf.fit(X_train, y_train)
    
    # View best hyperparameters
    #print('Best Penalty:', best_model.best_estimator_.get_params()['penalty'])
    #print('Best C:', best_model.best_estimator_.get_params()['C'])
    
    # best hyper parameters
    print('best_hyper_parameters:',best_model.best_params_)

    # performance:
    y_test_pred = best_model.best_estimator_.predict(X_test)
    acc = accuracy_score(y_test,y_test_pred)
    #print('Accuracy:', acc)

    p_test_pred = best_model.best_estimator_.predict_proba(X_test) # prob of [0,1]
    p_test_pred = p_test_pred[:,1] # prob of 1    
    fp,tp,thresholds = roc_curve(y_test, p_test_pred, drop_intermediate=False)
    roc_auc = auc(fp,tp)
    #print('AUC:', roc_auc)

    precision = precision_score(y_test,y_test_pred)
    #print('Precision:',precision)

    recall = recall_score(y_test,y_test_pred)
    #print('Recall:',recall)
    
    return acc,roc_auc,precision,recall

In [6]:
n_data = len(data_list)
roc_auc = np.zeros(n_data)   ; acc = np.zeros(n_data)
precision = np.zeros(n_data) ; recall = np.zeros(n_data)

#data_id = 0
for data_id in range(n_data):
    X_train,X_test,y_train,y_test = read_data(data_id)
    acc[data_id],roc_auc[data_id],precision[data_id],recall[data_id] =\
           measure_performance(X_train,X_test,y_train,y_test)
    print(data_id,acc[data_id],roc_auc[data_id])      

data_name: 1paradox
(array([0., 1.]), array([169,  60]))
(array([0., 1.]), array([60, 60]))
best_hyper_parameters: {'C': 100.0, 'penalty': 'l1', 'solver': 'liblinear'}
0 0.8166666666666667 0.880728554641598
data_name: 2peptide
(array([0., 1.]), array([675,  23]))
(array([0., 1.]), array([23, 23]))
best_hyper_parameters: {'C': 1.0, 'penalty': 'l1', 'solver': 'liblinear'}
1 1.0 1.0
data_name: 3stigma
(array([0., 1.]), array([2725, 7940]))
(array([0., 1.]), array([2725, 2725]))
best_hyper_parameters: {'C': 10.0, 'penalty': 'l1', 'solver': 'liblinear'}
2 1.0 1.0
data_name: 4nki
(array([0., 1.]), array([195,  77]))
(array([0., 1.]), array([77, 77]))
best_hyper_parameters: {'C': 100.0, 'penalty': 'l1', 'solver': 'liblinear'}
3 0.7532467532467533 0.8319838056680161
data_name: 5mental
(array([0., 1.]), array([616, 147]))
(array([0., 1.]), array([147, 147]))
best_hyper_parameters: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
4 0.6938775510204082 0.7645316549426139
data_name: 6smoking
(arr

/home/tai/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/home/tai/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


best_hyper_parameters: {'C': 10.0, 'penalty': 'l1', 'solver': 'liblinear'}
23 0.7313432835820896 0.8141003303458176
data_name: 29parkinson
(array([0., 1.]), array([ 48, 147]))
(array([0., 1.]), array([48, 48]))
best_hyper_parameters: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
24 0.6875 0.8620689655172414
data_name: 30paradox2
(array([0., 1.]), array([184,  52]))
(array([0., 1.]), array([52, 52]))
best_hyper_parameters: {'C': 1.0, 'penalty': 'l1', 'solver': 'liblinear'}
25 1.0 1.0
data_name: 31renal
(array([0., 1.]), array([197,  47]))
(array([0., 1.]), array([47, 47]))
best_hyper_parameters: {'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'}
26 0.7021276595744681 0.7454212454212454
data_name: 33svr
(array([0., 1.]), array([ 41, 186]))
(array([0., 1.]), array([41, 41]))
best_hyper_parameters: {'C': 1.0, 'penalty': 'l1', 'solver': 'liblinear'}
27 1.0 1.0
data_name: 35pcos
(array([0., 1.]), array([364, 177]))
(array([0., 1.]), array([177, 177]))
best_hyper_parameters: {'C': 1.0, '

In [7]:
print('acc_mean:',acc.mean())
print('roc_mean:',roc_auc.mean())

acc_mean: 0.8303560603766325
roc_mean: 0.8844472802354008


In [9]:
print('precision:',precision.mean())
print('recall:',recall.mean())

precision: 0.8643504488881772
recall: 0.7927957232246221


In [10]:
np.savetxt('LR_result.dat',(roc_auc,acc,precision,recall),fmt='%f')